In [2]:
import pandas as pd

In [7]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

In [8]:
novel = pd.read_csv("../datasets/classified_novel1.csv")

# 같은 title을 가진 데이터에서 100개씩 묶어 그룹화
novel["group"] = novel.groupby("title").cumcount() // 100  # 100개 단위 그룹 생성

novel_final = novel.groupby(["file_id", "group"]).agg({
    "title": "first",  # 대표 file_id
    "text": list,  # 100개씩 묶음
    "sentiment": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment에서 두 개의 감정을 각각의 칼럼으로 분리
novel_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(novel_final['sentiment'].tolist(), index=novel_final.index)

# 불필요한 sentiment 칼럼 삭제
novel_final = novel_final.drop(columns=["sentiment", "group", "file_id"])

novel_final.head()

,title,text,top_sentiment,second_sentiment
0,이야기꾼 구연설화,"[01범보다 무서운 곶감, 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 ...",분노,당황
1,이야기꾼 구연설화,"[<봄꿩은 제 울음에 저 죽는다>, 그 말과 같아서 사램이 잘못 되머넌 하는 말여....",상처,분노
2,이야기꾼 구연설화,"[그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어., “잡수라.”구., 그래 한 때를 몽...",상처,당황
3,이야기꾼 구연설화,"[그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,, “그러시냐구. 그...",상처,분노
4,이야기꾼 구연설화,"[나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, 거...",상처,분노


In [9]:
# 두 번째 감정이 없으면 첫 번재랑 같게
novel_final["second_sentiment"] = novel_final["second_sentiment"].fillna(novel_final["top_sentiment"])

In [10]:
novel_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             8972 non-null   object
 1   text              8972 non-null   object
 2   top_sentiment     8972 non-null   object
 3   second_sentiment  8972 non-null   object
dtypes: object(4)
memory usage: 280.5+ KB


In [11]:
novel_final.to_csv("../datasets/novel_final.csv")